# 9. 변형 (Mutation)
참고자료
 - https://medium.com/@meghamohan/mutable-and-immutable-side-of-python-c2145cf72747
 - https://medium.com/@birnbera/objects-memory-and-mutation-in-python-810bf090b63c

## Python 기본 자료형의 변형 가능 여부

| Class | Mutable  |
|:---:|:---:|
| bool | X |
| int | X |
| float | X |
| list | O |
| tuple | X |
| str | X |
| set | O |
| frozenset | X |
| dict | O |

## 예시
### List

In [1]:
foo = ['hi']
print(foo)   # ['hi']

['hi']


bar에 'bye'를 추가했지만 bar와 foo가 동일한 주소를 참조하고 있어 foo의 참조 값도 같이 변경

In [2]:
bar = foo
bar += ['bye']
print(foo)   # ['hi', 'bye']

['hi', 'bye']


In [3]:
print(hex(id(foo)), type(foo))   # 0x24a5d352fc8 <class 'list'>
print(hex(id(bar)), type(bar))   # 0x24a5d352fc8 <class 'list'>
print(foo == bar)   # True

0x24a5d352fc8 <class 'list'>
0x24a5d352fc8 <class 'list'>
True


### Integer
int도 유사하지만 약간의 차이가 있다

예시) 10이라는 정수 자체가 이미 메모리에 할당돼 있어 x는 해당 주소를 참조하는 형태

In [4]:
x = 10
y = x

print(id(x) == id(y))   # True
print(id(x) == id(10))   # True
print(id(y) == id(10))   # True
print(hex(id(10)))   # 0x7459b200

True
True
True
0x7459b200


따라서 x += 1을 해줄 경우 x의 주소가 변하는 것처럼 결과가 나오는데 이는 사실 x가 11을 참조하기 때문

In [5]:
x += 1

print(id(x) == id(y))   # False
print(id(x) == id(11))   # True
print(id(y) == id(10))   # True
print(hex(id(10)))   # 0x7459b200
print(hex(id(11)))   # 0x7459b220

False
True
True
0x7459b200
0x7459b220


In [6]:
y += 1

print(id(x) == id(y))   # True
print(id(x) == id(11))   # True
print(hex(id(11)))   # 0x7459b220

True
True
0x7459b220


### Tuple은 과연 불변할까?
Tuple은 파이썬의 대표적인 불변 자료형인데 Tuple의 요소는 충분히 변형 가능하다

In [7]:
s = ('holberton', [1, 2, 3])
t = s
s += (5,)
print(t)   # ('holberton', [1, 2, 3])
print(id(s) == id(t))   # False

('holberton', [1, 2, 3])
False


In [8]:
s[1] += [5,]   # Error

TypeError: 'tuple' object does not support item assignment

하지만 실제 s\[1\]이 참조하고 있는 list에는 새 요소가 추가됐으며 t도 동일한 주소를 참조하고 있기 때문에 t에도 반영

In [9]:
print(s)   # ('holberton', [1, 2, 3, 5], 5)
print(t)   # ('holberton', [1, 2, 3, 5])
print(hex(id(s[1])))   # 0x24a5cf51e48
print(id(s) == id(t))   # False

('holberton', [1, 2, 3, 5], 5)
('holberton', [1, 2, 3, 5])
0x24a5cf51e48
False


### 참조에 의한 호출
참조(변형 가능한) 객체를 함수의 인자로 넘기기 때문에 실제로 함수 내에서 이뤄진 연산이 해당 객체에 반영

In [10]:
def updateList(list1):
    list1 += [10]

n = [5, 6]
print(hex(id(n)))   # 0x24a5d4094c8
updateList(n)
print(n)   # [5, 6, 10]
print(hex(id(n)))   # 0x24a5d4094c8

0x24a5d4094c8
[5, 6, 10]
0x24a5d4094c8


### 값에 의한 호출
하지만 아래와 같은 경우 객체가 참조하고 있는 값만을 넘겨주기 때문에 해당 함수를 호출하는 동안에만 참조 객체가 변경

In [11]:
def updateNumber(n):
    tmp = hex(id(n))
    n += 10
    print(tmp, n, hex(id(n)))

b = 5
updateNumber(b)   # 0x7459b160 15 0x7459b2a0
print(b)   # 5

0x7459b160
0x7459b160 15 0x7459b2a0
5


### 기타

257 이상의 숫자는 동적으로 메모리에 할당

In [23]:
a = 257
print(hex(id(a)))   # 0x24a5d3fcfb0
b = 257
print(id(b) == id(a))   # False
print(id(b) == id(257))   # False

0x24a5d3fcfb0
False
False


정수는 32 byte C struct array에 저장

In [24]:
id(10) - id(9) == id(9) - id(8) == id(8) - id(7) == 32   # True

True

In [25]:
b = a
id(b) == id(a)   # True

True

하지만 257 이상의 숫자도 정수형이기 때문에 연산을 할 경우 새 객체를 생성

In [26]:
print(hex(id(b)))   # 0x24a5d3fcfb0
b += 1
print(b, hex(id(b)))   # 258 0x24a5d326ad0

0x24a5d3fcfb0
258 0x24a5d1ef470


동일하게 두 list의 요소를 추가하는 코드라도 inline 함수와 풀어서 쓰는 경우 차이가 있다

In [27]:
a = [1, 2]
print(hex(id(a)))   # 0x24a5d46bbc8

a = a + a
print(a, hex(id(a)))   # [1, 2, 1, 2] 0x24a5d46bd48

a *= 2
print(a, hex(id(a)))   # [1, 2, 1, 2, 1, 2, 1, 2] 0x24a5d46bd48

a += a
print(a, hex(id(a)))   # [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2] 0x24a5d46bd48

0x24a5d46bbc8
[1, 2, 1, 2] 0x24a5d46bd48
[1, 2, 1, 2, 1, 2, 1, 2] 0x24a5d46bd48
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2] 0x24a5d46bd48


문자열의 경우도 특정 길이 이하의 문자열에 대해서는 정수와 동일한 방식으로 저장 및 참조

In [16]:
s1 = "A random word"
s2 = "A random word"
s1 == s2   # False

False

In [17]:
s1 = "Arandomword"
s2 = "Arandomword"
s1 == s2   # True

True